<a href="https://colab.research.google.com/github/alirempel/cap-comp215/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2D Circle Simulation
### Author: Alison R
### Date: 2024-02-05

ADD DESCRIPTION OF PROJECT HEREEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE

Below are the imported modules and the constant variables.

In [1]:
import matplotlib.pyplot as plt
import random
import math
import itertools


LEFT = -1
RIGHT = 1
UP = 1
DOWN = -1

###Below are the customizable variables:
* *window_size* is the size you would like the window of the simulation to be (must be a TUPLE containing 2 INTs)
* *num_circles* is the number of circles you would like in the simulation (must be an INT)

  **IMPORTANT NOTE: The below variables are lists, and the length must be the same as num_circles**
* *circle_radii* is a list of the sizes of each circle's radius (each entry must be an INT)
* *circle_positions* is a list of the starting positions for each circle (each entry must be a **TUPLE** with 2 INTs and within the window size)
* *circle_directions* is a list containing the starting directions you want each circle to move- each entry must be a LIST containing two values:
  * First value must be either LEFT or RIGHT (*case sensitive*)
  * Second value must be either UP or DOWN (*also case sensitive*)



In [52]:
window_size = (500,500)
x,y = window_size

num_circles = 3

circle_radii = [10,5,20]
circle_positions = [(11,489),(23,100),(300,200)]
circle_directions = [[LEFT,UP],[RIGHT,DOWN],[LEFT,DOWN]]




In [32]:

######################################## ASK ABOUT USING OLD CLASS FROM CLASS !!!!!!!!!!!!!!!!!!!!!!!!!!!!



class Circle:
    """ Representation for the data and operations on a circle """

    def __init__(self, radius:float, centre:tuple, direction):
      assert len(centre) == 2
      self.radius = radius
      self.centre = centre
      self.direction = direction

    def __str__(self):
      """ Show the radius and position in a pretty way """
      return f"""Radius: {self.radius}, Position: {self.centre}"""

    def move(self):
        """ Move this circle by given (x,y) offset """
        self.centre = (self.centre[0] + self.direction[0], self.centre[1] + self.direction[1])

    def distance(self, other):
        """ Return the distance between this circle's centre and the other one """
        return math.dist(self.centre, other.centre)

    def intersects(self, other):
        """ Return True iff this circle intersects the other one """
        return self.radius + other.radius > self.distance(other)

In [4]:
# Test Circle class
def test_circle():
  c1 = Circle(3,(2,4))
  c2 = Circle(2,(5,8))
  assert c1 != c2 and c1.radius != c2.radius and c1.centre != c2.centre

  assert c1.distance(c2) == c2.distance(c1)
  assert c1.distance(c2) == 5.0
  assert c1.intersects(c2) == False
  c1.move((1,1))
  assert c1.intersects(c2) == True

  print('tests passed!!')

test_circle()

tests passed!!


In [54]:

class Simulation:
  """ Object that contains a number of circles that manages methods between the circles """

  def __init__(self,circles:list,window:tuple):
    self.x,self.y = window
    self.circles = circles
    assert len(window) == 2 and type(window[0]) == int and type(window[1]) == int

  def move_circles(self):
    """ Loop through circle list and move each one
    Switch the direction (x or y or both) of any circle that is hitting the edge of window """
    for circle in self.circles:

      if (circle.centre[0] + circle.radius) > x and circle.direction[0] == RIGHT:
        circle.direction[0] = LEFT

      if (circle.centre[0] - circle.radius) < 0 and circle.direction[0] == LEFT:
        circle.direction[0] = RIGHT

      if (circle.centre[1] + circle.radius) > y and circle.direction[1] == UP:
        circle.direction[1] = DOWN

      if (circle.centre[1] - circle.radius) < 0 and circle.direction[1] == DOWN:
        circle.direction[1] = UP

      circle.move()

      print(circle.centre) ####################### REMOVE LATER

  def check_collision(self):
    """ Loop through each unique pair of circles in circle list and check for collisions
    If there is a collision, the directions of the circles get swapper with each other's """

    for circle_pair in itertools.combinations(self.circles,2):
      if circle_pair[0].intersects(circle_pair[1]):
        circle_pair[0].direction = circle_pair[1]
        circle_pair[1].direction = circle_pair[0].direction

  def run_sim(self):
    """ Run the move_circles and check_collisions functions """
    count = 0                     ################# REMOVE
    running = True
    while running:
      self.move_circles()
      self.check_collision()

      count += 1         ############# REMOVE FROM HERE
      print("")
      if count > 10:            #################### TO HERE
        running = False


circle_list = [(Circle(rad,pos,dir)) for rad,pos,dir in zip(circle_radii,circle_positions,circle_directions)]

a = Simulation(circle_list,(100,200))
a.run_sim()

(12, 488)
(24, 99)
(299, 199)

(13, 487)
(25, 98)
(298, 198)

(14, 486)
(26, 97)
(297, 197)

(15, 485)
(27, 96)
(296, 196)

(16, 484)
(28, 95)
(295, 195)

(17, 483)
(29, 94)
(294, 194)

(18, 482)
(30, 93)
(293, 193)

(19, 481)
(31, 92)
(292, 192)

(20, 480)
(32, 91)
(291, 191)

(21, 479)
(33, 90)
(290, 190)

(22, 478)
(34, 89)
(289, 189)

